# Python `with` Statement Tutorial

This notebook explains the `with` statement in Python, context managers, and their practical applications.

When to Use Context Managers for I/O:
+ File operations
+ Database connections
+ Network sockets
+ Lock acquisition
+ Temporary resource management
+ Graphics contexts
+ Stream handling
+ HTTP connections


## 1. Basic File Handling Example

In [1]:
# Traditional way (without with)
print("Traditional way:")
try:
    f = open('example.txt', 'w')
    f.write('Hello, World!')
finally:
    f.close()

# Using with statement
print("\nUsing with statement:")
with open('example.txt', 'r') as f:
    content = f.read()
    print(f"Content: {content}")

# ! File is automatically closed after with block
print(f"Is file closed? {f.closed}")

Traditional way:

Using with statement:
Content: Hello, World!
Is file closed? True


## 2. Multiple Context Managers

In [2]:
# Using multiple files with with statement
with open('source.txt', 'w') as source:
    source.write('Some text to copy')

with open('source.txt', 'r') as source, open('destination.txt', 'w') as destination:
    content = source.read()
    destination.write(content)

# Read the destination file to verify
with open('destination.txt', 'r') as f:
    print(f"Destination content: {f.read()}")

Destination content: Some text to copy


## 3. Custom Context Manager using Class

In [ ]:
class Timer:
    def __init__(self, name):
        self.name = name
        
    def __enter__(self):
        import time
        print(f"Starting {self.name}...")
        self.start_time = time.time()
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        import time
        end_time = time.time()
        print(f"{self.name} took {end_time - self.start_time:.2f} seconds")

# Using our custom context manager
with Timer("Operation"):
    # Simulate some work
    import time
    time.sleep(1)

## 4. Custom Context Manager using contextlib

In [ ]:
from contextlib import contextmanager

@contextmanager
def indentation(spaces):
    try:
        # Setup
        original_print = print
        def indented_print(*args, **kwargs):
            original_print(' ' * spaces, *args, **kwargs)
        builtins = __import__('builtins')
        builtins.print = indented_print
        
        yield  # This is where the with block's code runs
        
    finally:
        # Cleanup
        builtins.print = original_print

# Using our indentation context manager
print("Normal print")
with indentation(4):
    print("Indented print")
print("Back to normal")

## 5. Error Handling in Context Managers

In [ ]:
class ErrorHandler:
    def __enter__(self):
        print("Entering the context")
        return self
    
    def __exit__(self, exc_type, exc_value, traceback):
        if exc_type is not None:
            print(f"An error occurred: {exc_type.__name__}: {exc_value}")
            return True  # Suppress the error
        print("Exiting the context normally")

# Test with no error
print("Test 1: No error")
with ErrorHandler():
    print("Doing something normal")

# Test with error
print("\nTest 2: With error")
with ErrorHandler():
    raise ValueError("Something went wrong!")

print("\nCode continues to run")

## 6. Practical Example: Database Connection

In [ ]:
class DatabaseConnection:
    def __init__(self, db_name):
        self.db_name = db_name
        
    def __enter__(self):
        print(f"Connecting to database {self.db_name}...")
        # Simulate connection setup
        self.data = {}
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        print(f"Closing connection to {self.db_name}")
        # Cleanup would happen here
        self.data = None
    
    def query(self, sql):
        print(f"Executing: {sql}")
        return ["result1", "result2"]

# Using the database connection
with DatabaseConnection("example_db") as db:
    results = db.query("SELECT * FROM table")
    print(f"Results: {results}")

# ! Connection is automatically closed here

## 7. Nested Context Managers

In [ ]:
class Level:
    def __init__(self, level):
        self.level = level
    
    def __enter__(self):
        print(f"Entering level {self.level}")
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        print(f"Exiting level {self.level}")

# Using nested context managers
with Level(1):
    print("In level 1")
    with Level(2):
        print("In level 2")
        with Level(3):
            print("In level 3")

## 8. Context Manager for Temporary File

In [ ]:
import os
from contextlib import contextmanager

@contextmanager
def temporary_file(content):
    # Setup
    filename = 'temp.txt'
    with open(filename, 'w') as f:
        f.write(content)
    try:
        yield filename
    finally:
        # Cleanup
        if os.path.exists(filename):
            os.remove(filename)

# Using the temporary file
with temporary_file("Hello, World!") as filename:
    print(f"File exists during with block: {os.path.exists(filename)}")
    with open(filename, 'r') as f:
        print(f"Content: {f.read()}")

print(f"File exists after with block: {os.path.exists(filename)}")

## Cleanup

In [3]:
import os

# Clean up files created during the tutorial
files_to_remove = ['example.txt', 'source.txt', 'destination.txt']

for file in files_to_remove:
    if os.path.exists(file):
        os.remove(file)
        print(f"Removed {file}")

Removed example.txt
Removed source.txt
Removed destination.txt
